## test data에도 같은 id를 사용한다면 id변수를 encoding하여 모델학습에 참여시키는 것이 좋음
- target encoding하여 새로운 변수를 형성
- 새로운변수의 target과의 상관은 0.2 근처
- 다른 수치형 변수들의 상관과 비교해도 유의미한 변수로 바뀜
- 과적합 방지를 위한 fold 진행하였음 

In [9]:
import pandas as pd

# CSV 파일을 불러와서 df_concat이라는 DataFrame으로 저장
df_concat = pd.read_csv('df_concat_clear.csv')

# 불러온 데이터 확인 (옵션)
print(df_concat.head())

  Wip Line_Dam Process Desc._Dam     Equipment_Dam Model.Suffix_Dam  \
0      IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334501   
1      IVI-OB6     Dam Dispenser  Dam dispenser #2      AJX75334501   
2      IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334501   
3      IVI-OB6     Dam Dispenser  Dam dispenser #2      AJX75334501   
4      IVI-OB6     Dam Dispenser  Dam dispenser #1      AJX75334501   

  Workorder_Dam  Insp. Seq No._Dam Insp Judge Code_Dam  \
0    3G1XB278-1                  1                  OK   
1    3I1XB274-1                  1                  OK   
2    3G1X8298-1                  1                  OK   
3    4B1X9671-2                  1                  OK   
4    3J1XC129-1                  1                  OK   

   CURE END POSITION X Collect Result_Dam  CURE END POSITION X Unit Time_Dam  \
0                                   240.0                                NaN   
1                                  1000.0                       

In [10]:
# 특정 변수의 고유값 비율 확인
value_counts = df_concat['target'].value_counts(normalize=True) * 100

# 결과 출력
print(value_counts)

target
Normal      50.0
AbNormal    50.0
Name: proportion, dtype: float64


## target encoding 하였음


In [11]:
import pandas as pd
from sklearn.model_selection import KFold



# target 변수를 숫자로 변환 (Normal -> 0, AbNormal -> 1)
df_concat['target_numeric'] = df_concat['target'].map({'Normal': 0, 'AbNormal': 1})

# KFold 설정: 5개의 폴드로 데이터를 나눔
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 새로운 열을 생성하여 Target Encoding 값을 저장
df_concat['Workorder_Fill2_Target_Encoded_CV'] = 0

# KFold를 이용한 Target Encoding 수행
for train_idx, val_idx in kf.split(df_concat):
    # 학습 데이터와 검증 데이터를 분리
    train_df, val_df = df_concat.iloc[train_idx], df_concat.iloc[val_idx]
    
    # 학습 데이터에서 Workorder_Fill2에 따른 target 평균 계산
    target_mean = train_df.groupby('Workorder_Fill2')['target_numeric'].mean()
    
    # 검증 데이터에 대해 평균값을 매핑
    df_concat.loc[val_idx, 'Workorder_Fill2_Target_Encoded_CV'] = val_df['Workorder_Fill2'].map(target_mean)

# 결과 확인 (예: df_concat.head())
print(df_concat[['Workorder_Fill2', 'target_numeric', 'Workorder_Fill2_Target_Encoded_CV']].head())

# 생성된 데이터프레임을 저장하거나 다음 단계로 활용
# df_concat.to_csv('encoded_data.csv', index=False)


  Workorder_Fill2  target_numeric  Workorder_Fill2_Target_Encoded_CV
0      3G1XB278-1               0                           0.818182
1      3I1XB274-1               0                           0.250000
2      3G1X8298-1               0                           0.562500
3      4B1X9671-2               0                           0.400000
4      3J1XC129-1               0                           0.750000


C:\Users\lee seung woo\AppData\Local\Temp\ipykernel_4848\3750365904.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.5        0.5        0.6        0.         0.57142857 0.25
 0.4        0.69230769 1.         0.23076923 0.         0.625
 0.5        0.625      0.5        0.5        0.         0.42857143
 0.33333333 0.23529412 0.         0.5        0.375      0.5
 0.63636364 0.33333333 0.         0.         0.6        0.55555556
 0.23529412 0.75       0.75       0.21428571 0.         0.55555556
 0.71428571 0.625      0.27272727 0.75       0.42857143 0.
 0.375      0.5        0.66666667 0.33333333 0.5               nan
 0.33333333 0.7        0.38095238 0.625      0.5        0.75
 0.5        0.41176471 0.57142857 0.35714286 0.25       0.
 0.83333333 0.54545455 0.5        0.                nan 0.625
        nan 0.16666667 1.         0.5        0.5        0.5
 0.71428571 0.42857143 0.33333333 0.33333333 0.        

## 새로운 변수 결측치없이 잘 encoding 됨

In [12]:
# 특정 변수의 고유값 비율 확인
value_counts = df_concat['Workorder_Fill2_Target_Encoded_CV'].value_counts(normalize=True) * 100

# 결과 출력
print(value_counts)

Workorder_Fill2_Target_Encoded_CV
0.500000    11.026699
0.000000     8.921207
0.333333     6.294769
0.666667     4.753636
1.000000     4.536575
              ...    
0.181818     0.043412
0.923077     0.043412
0.238095     0.021706
0.388889     0.021706
0.176471     0.021706
Name: proportion, Length: 130, dtype: float64


In [13]:
correlation = df_concat['Workorder_Fill2_Target_Encoded_CV'].corr(df_concat['target_numeric'])
print(f"Correlation between Target Encoded variable and target: {correlation}")


Correlation between Target Encoded variable and target: 0.21278301334820898


## 다른 수치형 변수와 비교하더라도 target과의 가장 높은 상관관계 형성(0.21) 
- target encoding 특성상 오버피팅만 경계하면됨

In [17]:
# 숫자형 변수만 선택
numeric_cols = df_concat.select_dtypes(include=['float64', 'int64'])

# target 변수와의 상관관계 계산
correlation_with_target = numeric_cols.corrwith(df_concat['target_numeric'])

# 결과 출력
print(correlation_with_target.sort_values(ascending=False))


target_numeric                                    1.000000
Workorder_Fill2_Target_Encoded_CV                 0.212783
Stage2 Line1 Distance Speed Collect Result_Dam    0.135850
Stage2 Line2 Distance Speed Collect Result_Dam    0.135526
Stage2 Line4 Distance Speed Collect Result_Dam    0.135526
                                                    ...   
Receip No Unit Time_Fill2                              NaN
Receip No Judge Value_Fill2                            NaN
WorkMode Collect Result_Fill2                          NaN
WorkMode Unit Time_Fill2                               NaN
WorkMode Judge Value_Fill2                             NaN
Length: 432, dtype: float64


C:\Users\lee seung woo\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\lee seung woo\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


## 다른 범주형 변수에 대해서도 id 많은 변수들 test_data와의 중복성 체크해보고 

- 가치가 있다고 판단이 되는 변수면 target encoding 진행해보자

In [18]:
import pandas as pd
from sklearn.model_selection import KFold



# target 변수를 숫자로 변환 (Normal -> 0, AbNormal -> 1)
df_concat['target_numeric'] = df_concat['target'].map({'Normal': 0, 'AbNormal': 1})

# KFold 설정: 5개의 폴드로 데이터를 나눔
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 새로운 열을 생성하여 Target Encoding 값을 저장
df_concat['Workorder_Dam_Target_Encoded_CV'] = 0

# KFold를 이용한 Target Encoding 수행
for train_idx, val_idx in kf.split(df_concat):
    # 학습 데이터와 검증 데이터를 분리
    train_df, val_df = df_concat.iloc[train_idx], df_concat.iloc[val_idx]
    
    # 학습 데이터에서 Workorder_Fill2에 따른 target 평균 계산
    target_mean = train_df.groupby('Workorder_Dam')['target_numeric'].mean()
    
    # 검증 데이터에 대해 평균값을 매핑
    df_concat.loc[val_idx, 'Workorder_Dam_Target_Encoded_CV'] = val_df['Workorder_Dam'].map(target_mean)

# 결과 확인 (예: df_concat.head())
print(df_concat[['Workorder_Dam', 'target_numeric', 'Workorder_Dam_Target_Encoded_CV']].head())

# 생성된 데이터프레임을 저장하거나 다음 단계로 활용
# df_concat.to_csv('encoded_data.csv', index=False)


  Workorder_Dam  target_numeric  Workorder_Dam_Target_Encoded_CV
0    3G1XB278-1               0                         0.818182
1    3I1XB274-1               0                         0.250000
2    3G1X8298-1               0                         0.562500
3    4B1X9671-2               0                         0.400000
4    3J1XC129-1               0                         0.750000


C:\Users\lee seung woo\AppData\Local\Temp\ipykernel_4848\1493533542.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.5        0.5        0.6        0.         0.57142857 0.25
 0.4        0.69230769 1.         0.23076923 0.         0.625
 0.5        0.625      0.5        0.5        0.         0.42857143
 0.33333333 0.23529412 0.         0.5        0.375      0.5
 0.63636364 0.33333333 0.         0.         0.6        0.55555556
 0.23529412 0.75       0.75       0.21428571 0.         0.55555556
 0.71428571 0.625      0.27272727 0.75       0.42857143 0.
 0.375      0.5        0.66666667 0.33333333 0.5               nan
 0.33333333 0.7        0.38095238 0.625      0.5        0.75
 0.5        0.41176471 0.57142857 0.35714286 0.25       0.
 0.83333333 0.54545455 0.5        0.                nan 0.625
        nan 0.16666667 1.         0.5        0.5        0.5
 0.71428571 0.42857143 0.33333333 0.33333333 0.        

In [20]:
# 특정 변수의 고유값 비율 확인
value_counts = df_concat['Workorder_Dam_Target_Encoded_CV'].value_counts(normalize=True) * 100

# 결과 출력
print(value_counts)

Workorder_Dam_Target_Encoded_CV
0.500000    11.026699
0.000000     8.921207
0.333333     6.294769
0.666667     4.753636
1.000000     4.536575
              ...    
0.181818     0.043412
0.923077     0.043412
0.238095     0.021706
0.388889     0.021706
0.176471     0.021706
Name: proportion, Length: 130, dtype: float64


In [21]:
# 숫자형 변수만 선택
numeric_cols = df_concat.select_dtypes(include=['float64', 'int64'])

# target 변수와의 상관관계 계산
correlation_with_target = numeric_cols.corrwith(df_concat['target_numeric'])

# 결과 출력
print(correlation_with_target.sort_values(ascending=False))

target_numeric                                    1.000000
Workorder_Dam_Target_Encoded_CV                   0.212783
Workorder_Fill2_Target_Encoded_CV                 0.212783
Stage2 Line1 Distance Speed Collect Result_Dam    0.135850
Stage2 Line4 Distance Speed Collect Result_Dam    0.135526
                                                    ...   
Receip No Unit Time_Fill2                              NaN
Receip No Judge Value_Fill2                            NaN
WorkMode Collect Result_Fill2                          NaN
WorkMode Unit Time_Fill2                               NaN
WorkMode Judge Value_Fill2                             NaN
Length: 433, dtype: float64


C:\Users\lee seung woo\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\lee seung woo\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


##  Model.Suffix_Fill1_변수는 상관이 낮아 target_encoding 하는 것이 의미가 없음

In [22]:
import pandas as pd
from sklearn.model_selection import KFold



# target 변수를 숫자로 변환 (Normal -> 0, AbNormal -> 1)
df_concat['target_numeric'] = df_concat['target'].map({'Normal': 0, 'AbNormal': 1})

# KFold 설정: 5개의 폴드로 데이터를 나눔
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# 새로운 열을 생성하여 Target Encoding 값을 저장
df_concat['Model.Suffix_Fill1_Target_Encoded_CV'] = 0

# KFold를 이용한 Target Encoding 수행
for train_idx, val_idx in kf.split(df_concat):
    # 학습 데이터와 검증 데이터를 분리
    train_df, val_df = df_concat.iloc[train_idx], df_concat.iloc[val_idx]
    
    # 학습 데이터에서 Workorder_Fill2에 따른 target 평균 계산
    target_mean = train_df.groupby('Model.Suffix_Fill1')['target_numeric'].mean()
    
    # 검증 데이터에 대해 평균값을 매핑
    df_concat.loc[val_idx, 'Model.Suffix_Fill1_Target_Encoded_CV'] = val_df['Model.Suffix_Fill1'].map(target_mean)
  
# 결과 확인 (예: df_concat.head())
print(df_concat[['Model.Suffix_Fill1', 'target_numeric', 'Model.Suffix_Fill1_Target_Encoded_CV']].head())

# 생성된 데이터프레임을 저장하거나 다음 단계로 활용
# df_concat.to_csv('encoded_data.csv', index=False)


  Model.Suffix_Fill1  target_numeric  Model.Suffix_Fill1_Target_Encoded_CV
0        AJX75334501               0                              0.493312
1        AJX75334501               0                              0.491290
2        AJX75334501               0                              0.491290
3        AJX75334501               0                              0.492140
4        AJX75334501               0                              0.492140


C:\Users\lee seung woo\AppData\Local\Temp\ipykernel_4848\2788930857.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.49454429 0.49454429 0.49454429 0.55820896 0.49454429 0.49454429
 0.49454429 0.49454429 0.49454429 0.49454429 0.49454429 0.55820896
 0.55820896 0.49454429 0.49454429 0.49454429 0.49454429 0.49454429
 0.49454429 0.49454429 0.49454429 0.49454429 0.49454429 0.49454429
 0.49454429 0.49454429 0.49454429 0.49454429 0.49454429 0.49454429
 0.49454429 0.49454429 0.49454429 0.49454429 0.49454429 0.49454429
 0.49454429 0.49454429 0.49454429 0.49454429 0.49454429 0.49454429
 0.49454429 0.49454429 0.49454429 0.49454429 0.55820896 0.49454429
 0.49454429 0.55820896 0.55820896 0.55820896 0.49454429 0.49454429
 0.49454429 0.49454429 0.55820896 0.49454429 0.49454429 0.64
 0.49454429 0.49454429 0.49454429 0.49454429 0.55820896 0.49454429
 0.49454429 0.49454429 0.49454429 0.49454429 0.55820896 0.49454429
 0.494544

In [23]:
correlation = df_concat['Model.Suffix_Fill1_Target_Encoded_CV'].corr(df_concat['target_numeric'])
print(f"Correlation between Target Encoded variable and target: {correlation}")


Correlation between Target Encoded variable and target: 0.07756926449049699


In [29]:
# 변수 선택
variable = 'target'

# 고유값과 비율 계산
value_counts = df_concat[variable].value_counts(normalize=True)  # normalize=True는 비율을 계산

# 결과 출력
print(f"'{variable}' 변수의 고유값과 비율:")
print(value_counts)

'target' 변수의 고유값과 비율:
target
Normal      0.5
AbNormal    0.5
Name: proportion, dtype: float64
